# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [11]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [12]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [13]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [14]:
# Write your code below.

shift_dd = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(close_lag = x['Close'].shift(1))
)

dd_feat = shift_dd.assign(
    returns = lambda x: x['Close']/x['close_lag'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

dd_feat

C:\Users\Kirby\AppData\Local\Temp\ipykernel_11424\3416363528.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  shift_dd = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,close_lag,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,object,int32,float64,float64,float64
AGRO,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [15]:
# Write your code below.

df = dd_feat.compute()

df['avg_10_day_return'] = (
    df.groupby('ticker')['returns'].rolling(10).mean().sort_index(level=1).values
    )

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, we could have done it while using a Dask Dataframe

+ Would it have been better to do it in Dask? Why?

No - since the dataset is relatively small - < 40 MB - pandas can just read the entire dataset in memory and do the calcations there.
With Dask, we'd have to pick out 10 rows per ticker (the index) and calculate the mean. Dask is efficient when maniulating data based off the index, not the features. 

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

# Was it necessary to convert to pandas to calculate the moving average return?

No, we could have done it while using a Dask Dataframe

# Would it have been better to do it in Dask? Why?

No - since the dataset is relatively small - < 40 MB - pandas can just read the entire dataset in memory and do the calcations there.
With Dask, we'd have to pick out 10 rows per ticker (the index) and calculate the mean. Dask is efficient when maniulating data based off the index, not the features. 

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.